In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 300
generations = int(3000/necoc)
generations

10

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
128


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:746
number of class2:854

 ****** Running generation 0 ****** 

Population's average fitness: 0.50146 stdev: 0.04333
Best fitness: 0.62375 - size: (2, 3) - species 3 - id 16
Average adjusted fitness: 0.151
Mean genetic distance 2.695, standard deviation 0.527
Population of 199 members in 14 species
Total extinctions: 0
Generation time: 3.017 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51879 stdev: 0.04929
Best fitness: 0.66500 - size: (3, 5) - species 3 - id 345
Average adjusted fitness: 0.175
Mean genetic distance 2.852, standard deviation 0.523
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 3.549 sec (3.283 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.52697 stdev: 0.05357
Best fitness: 0.67375 - size: (2, 4) - species 5 - id 404
Average adjusted fitness: 0.143
Mean genetic distance 2.834, standard deviation 0.575
Population of 198 members in 16 species
Total 

Population's average fitness: 0.53232 stdev: 0.05611
Best fitness: 0.67000 - size: (5, 8) - species 7 - id 858
Average adjusted fitness: 0.130
Mean genetic distance 3.023, standard deviation 0.549
Population of 205 members in 34 species
Total extinctions: 0
Generation time: 4.212 sec (3.867 average)

 ****** Running generation 5 ****** 

Population's average fitness: 0.53766 stdev: 0.05811
Best fitness: 0.68000 - size: (5, 8) - species 16 - id 993
Average adjusted fitness: 0.110
Mean genetic distance 3.047, standard deviation 0.528
Population of 204 members in 34 species
Total extinctions: 0
Generation time: 4.482 sec (3.969 average)

 ****** Running generation 6 ****** 

Population's average fitness: 0.54501 stdev: 0.05793
Best fitness: 0.68000 - size: (5, 8) - species 16 - id 993
Average adjusted fitness: 0.117
Mean genetic distance 3.045, standard deviation 0.531
Population of 198 members in 36 species
Total extinctions: 0
Generation time: 4.677 sec (4.071 average)

 ****** Running 

Population's average fitness: 0.54326 stdev: 0.05280
Best fitness: 0.66812 - size: (5, 8) - species 27 - id 1450
Average adjusted fitness: 0.171
Mean genetic distance 3.192, standard deviation 0.561
Population of 200 members in 31 species
Total extinctions: 0
Generation time: 4.568 sec (3.964 average)

 ****** Running generation 9 ****** 

Population's average fitness: 0.54550 stdev: 0.05319
Best fitness: 0.67000 - size: (6, 10) - species 27 - id 1613
Average adjusted fitness: 0.106
Mean genetic distance 3.236, standard deviation 0.572
Population of 196 members in 32 species
Total extinctions: 0
Generation time: 4.646 sec (4.032 average)
running:5/30
number of class1:975
number of class2:625

 ****** Running generation 0 ****** 

Population's average fitness: 0.49095 stdev: 0.09573
Best fitness: 0.64750 - size: (2, 3) - species 3 - id 67
Average adjusted fitness: 0.150
Mean genetic distance 2.720, standard deviation 0.519
Population of 200 members in 16 species
Total extinctions: 0
Gen

Population's average fitness: 0.59591 stdev: 0.12326
Best fitness: 0.69437 - size: (3, 5) - species 14 - id 322
Average adjusted fitness: 0.273
Mean genetic distance 2.868, standard deviation 0.559
Population of 199 members in 24 species
Total extinctions: 0
Generation time: 3.581 sec (3.267 average)

 ****** Running generation 3 ****** 

Population's average fitness: 0.59567 stdev: 0.12541
Best fitness: 0.69437 - size: (3, 5) - species 14 - id 322
Average adjusted fitness: 0.258
Mean genetic distance 2.853, standard deviation 0.568
Population of 202 members in 24 species
Total extinctions: 0
Generation time: 3.619 sec (3.355 average)

 ****** Running generation 4 ****** 

Population's average fitness: 0.60415 stdev: 0.11617
Best fitness: 0.69437 - size: (3, 5) - species 14 - id 322
Average adjusted fitness: 0.275
Mean genetic distance 2.888, standard deviation 0.569
Population of 202 members in 26 species
Total extinctions: 0
Generation time: 3.867 sec (3.458 average)

 ****** Running

Population's average fitness: 0.55528 stdev: 0.09256
Best fitness: 0.66875 - size: (2, 2) - species 11 - id 285
Average adjusted fitness: 0.191
Mean genetic distance 2.929, standard deviation 0.557
Population of 200 members in 20 species
Total extinctions: 0
Generation time: 4.433 sec (3.760 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.54469 stdev: 0.09784
Best fitness: 0.67563 - size: (4, 6) - species 17 - id 1353
Average adjusted fitness: 0.177
Mean genetic distance 2.914, standard deviation 0.559
Population of 197 members in 20 species
Total extinctions: 0
Generation time: 4.518 sec (3.855 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.55062 stdev: 0.09528
Best fitness: 0.67563 - size: (4, 6) - species 17 - id 1353
Average adjusted fitness: 0.166
Mean genetic distance 2.877, standard deviation 0.551
Population of 200 members in 20 species
Total extinctions: 0
Generation time: 4.451 sec (3.921 average)

 ****** Runni

...

Population's average fitness: 0.53774 stdev: 0.08865
Best fitness: 0.62125 - size: (2, 4) - species 3 - id 247
Average adjusted fitness: 0.167
Mean genetic distance 2.916, standard deviation 0.579
Population of 201 members in 20 species
Total extinctions: 0
Generation time: 3.211 sec (3.024 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.54408 stdev: 0.08665
Best fitness: 0.63062 - size: (3, 5) - species 2 - id 499
Average adjusted fitness: 0.165
Mean genetic distance 2.977, standard deviation 0.618
Population of 202 members in 21 species
Total extinctions: 0
Generation time: 3.385 sec (3.145 average)

 ****** Running generation 3 ****** 

Population's average fitness: 0.54785 stdev: 0.08681
Best fitness: 0.63062 - size: (3, 5) - species 5 - id 499
Average adjusted fitness: 0.183
Mean genetic distance 3.051, standard deviation 0.595
Population of 200 members in 26 species
Total extinctions: 0
Generation time: 3.828 sec (3.315 average)

 ****** Running ge

Population's average fitness: 0.55863 stdev: 0.08525
Best fitness: 0.63750 - size: (3, 5) - species 26 - id 550
Average adjusted fitness: 0.164
Mean genetic distance 3.040, standard deviation 0.536
Population of 205 members in 33 species
Total extinctions: 0
Generation time: 4.018 sec (3.485 average)

 ****** Running generation 6 ****** 

Population's average fitness: 0.56162 stdev: 0.08239
Best fitness: 0.63750 - size: (3, 5) - species 26 - id 550
Average adjusted fitness: 0.178
Mean genetic distance 3.027, standard deviation 0.530
Population of 203 members in 34 species
Total extinctions: 0
Generation time: 4.027 sec (3.562 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.55371 stdev: 0.08661
Best fitness: 0.63750 - size: (3, 5) - species 26 - id 550
Average adjusted fitness: 0.153
Mean genetic distance 3.013, standard deviation 0.539
Population of 205 members in 34 species
Total extinctions: 0
Generation time: 4.146 sec (3.635 average)

 ****** Running

Population's average fitness: 0.53416 stdev: 0.04582
Best fitness: 0.62500 - size: (7, 10) - species 10 - id 1339
Average adjusted fitness: 0.126
Mean genetic distance 3.076, standard deviation 0.512
Population of 203 members in 29 species
Total extinctions: 0
Generation time: 4.581 sec (4.377 average)
running:293/30
number of class1:863
number of class2:737

 ****** Running generation 0 ****** 

Population's average fitness: 0.49538 stdev: 0.04029
Best fitness: 0.59937 - size: (2, 3) - species 1 - id 149
Average adjusted fitness: 0.122
Mean genetic distance 2.657, standard deviation 0.532
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 2.786 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51160 stdev: 0.03929
Best fitness: 0.59937 - size: (2, 3) - species 1 - id 149
Average adjusted fitness: 0.142
Mean genetic distance 2.689, standard deviation 0.547
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 4

Population's average fitness: 0.52771 stdev: 0.04689
Best fitness: 0.64187 - size: (5, 9) - species 7 - id 730
Average adjusted fitness: 0.157
Mean genetic distance 2.815, standard deviation 0.589
Population of 198 members in 18 species
Total extinctions: 0
Generation time: 4.480 sec (3.952 average)

 ****** Running generation 4 ****** 

Population's average fitness: 0.52825 stdev: 0.04858
Best fitness: 0.64187 - size: (5, 9) - species 1 - id 730
Average adjusted fitness: 0.093
Mean genetic distance 2.798, standard deviation 0.546
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 4.451 sec (4.051 average)

 ****** Running generation 5 ****** 

Population's average fitness: 0.53111 stdev: 0.05318
Best fitness: 0.64187 - size: (5, 9) - species 1 - id 730
Average adjusted fitness: 0.139
Mean genetic distance 2.844, standard deviation 0.536
Population of 200 members in 19 species
Total extinctions: 0
Generation time: 4.813 sec (4.178 average)

 ****** Running ge

Population's average fitness: 0.54467 stdev: 0.05245
Best fitness: 0.67375 - size: (6, 9) - species 10 - id 1366
Average adjusted fitness: 0.118
Mean genetic distance 3.124, standard deviation 0.509
Population of 202 members in 31 species
Total extinctions: 0
Generation time: 5.241 sec (4.631 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.54698 stdev: 0.05245
Best fitness: 0.67375 - size: (6, 9) - species 10 - id 1366
Average adjusted fitness: 0.112
Mean genetic distance 3.106, standard deviation 0.520
Population of 201 members in 31 species
Total extinctions: 0
Generation time: 4.822 sec (4.653 average)

 ****** Running generation 9 ****** 

Population's average fitness: 0.54604 stdev: 0.05342
Best fitness: 0.67375 - size: (5, 9) - species 22 - id 1649
Average adjusted fitness: 0.109
Mean genetic distance 3.136, standard deviation 0.499
Population of 196 members in 31 species
Total extinctions: 0
Generation time: 5.810 sec (4.768 average)
running:298/3

In [8]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.52

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.595
Avg Base Learner:0.6645958333333333


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:2484 Total connections::3653
